# El tratamiento de las variables categóricas

Para trabajar con variables categoricas, generaremos variables dummy, siempre se puede hacer n-1 variables nuevas para los posibles valores de la categoria.

Es decir si tenemos por ej: sexo: hombre y mujer, basta hacer 1 varible dummy 
si tenemos distrito: 1,2,3 necesitamos hacer 2 variables dummy 

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("../datasets/ecom-expense/Ecom Expense.csv")

In [3]:
df.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485


**Queremos predecir los gastos totales considerando si es hombre o mjer, el barrio,etc**

In [4]:
dummy_gender = pd.get_dummies(df["Gender"],prefix="Gender")
dummy_city_tier = pd.get_dummies(df["City Tier"],prefix="City")

In [5]:
dummy_city_tier

,City_Tier 1,City_Tier 2,City_Tier 3
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0
...,...,...,...
2357,0,1,0
2358,0,1,0
2359,0,0,1
2360,1,0,0


In [6]:
dummy_gender

,Gender_Female,Gender_Male
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
2357,0,1
2358,0,1
2359,1,0
2360,1,0


In [7]:
column_names= df.columns.values.tolist()
column_names

['Transaction ID',
 'Age ',
 ' Items ',
 'Monthly Income',
 'Transaction Time',
 'Record',
 'Gender',
 'City Tier',
 'Total Spend']

In [8]:
df_new= df[column_names].join(dummy_gender)
column_names= df_new.columns.values.tolist()
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0


In [9]:
df_new= df_new[column_names].join(dummy_city_tier)
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0


In [10]:
features_cols =['Monthly Income',  'Transaction Time','Gender_Male',
                'Gender_Female','City_Tier 1','City_Tier 2','City_Tier 3',"Record"]

In [11]:
x= df_new[features_cols]
y= df_new['Total Spend']

In [12]:
lm = LinearRegression()
lm.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
print(lm.intercept_)
print(lm.coef_)

-79.41713030137271
[ 1.47538980e-01  1.54946125e-01  1.31025013e+02 -1.31025013e+02
  7.67643260e+01  5.51389743e+01 -1.31903300e+02  7.72233446e+02]


In [14]:
list(zip(features_cols,lm.coef_))

[('Monthly Income', 0.14753898049205744),
 ('Transaction Time', 0.15494612549589679),
 ('Gender_Male', 131.0250132555466),
 ('Gender_Female', -131.0250132555465),
 ('City_Tier 1', 76.76432601049541),
 ('City_Tier 2', 55.13897430923251),
 ('City_Tier 3', -131.90330031972795),
 ('Record', 772.2334457445644)]

In [15]:
lm.score(x,y)   # vemos que el modelo es flojito, el R^2 (0.19) es muy bajo, 
                #seguramente debido a que no utilizamos todas las variables. Veamos si agregamos Record

0.9179923586131016

El R^2 paso de 0.19 a 0.91 utilizando una columna más como record

EL modelo puede ser escrito como:

**Total Spend= -79.41713030137271+ 'Monthly Income'*0.14753898049205744 + 'Transaction Time'* 0.15494612549589679             +'Gender_Male' * 131.0250132555466 + 'Gender_Female'* -131.0250132555465 + City_Tier 1'*76.764 +
           +'City_Tier 2' * 55.13897430923251 + 'City_Tier 3' *-131.90330031972795 +'Record'*772.2334457445644 **


In [16]:
df_new["prediction"]= -79.41713030137271+df_new['Monthly Income']*0.14753898049205744   + df_new['Transaction Time']* 0.15494612549589679+ df_new['Gender_Male']*131.0250132555466 +df_new['Gender_Female']*(-131.0250132555465)+df_new['City_Tier 1']*76.76432601049541 +df_new['City_Tier 2']* 55.13897430923251 + df_new['City_Tier 3']*(-131.90330031972795) + df_new['Record']*772.2334457445644

In [17]:
### otra forma de calcular predicciones
df_new["prediction"]= lm.predict(pd.DataFrame(df_new[features_cols]))

In [18]:
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,prediction
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0,4903.696720
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0,4799.434826
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0,5157.082504
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0,8068.012996
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0,3581.980335


In [19]:
SSD = np.sum((df_new["prediction"]-df_new["Total Spend"])**2)

In [20]:
SSD

1517733985.3408163

In [21]:
RSE = np.sqrt(SSD/len(df_new)-len(features_cols)-1)

In [22]:
RSE

801.5947080211582

In [23]:
sales_mean= np.mean(df_new["Total Spend"])

In [24]:
sales_mean

6163.176415976714

In [25]:
error= RSE/sales_mean
print("el error del modelo es : ",error*100)

el error del modelo es :  13.006194434791704


Notar que hemos calculado el modelo global, sin embargo este se reduce cuando introducimos los valores de las variables, es decir:

* global : 

Total Spend= -79.41713030137271+ 'Monthly Income'0.14753898049205744 + 'Transaction Time'* 0.15494612549589679 +'Gender_Male' * 131.0250132555466 + 'Gender_Female'* -131.0250132555465 + City_Tier 1'76.764 + +'City_Tier 2' * 55.13897430923251 + 'City_Tier 3' *-131.90330031972795 +'Record'772.2334457445644 
* Si es hombre y vive en CT1: 

Total Spend= -79.41713030137271+ 'Monthly Income'0.14753898049205744 + 'Transaction Time'* 0.15494612549589679 + 131.0250132555466 + + City_Tier 1'76.764 +'Record'772.2334457445644 
* Si es hombre y vive en CT2: 

Total Spend= -79.41713030137271+ 'Monthly Income'0.14753898049205744 + 'Transaction Time'* 0.15494612549589679 +'Gender_Male' * 131.0250132555466 +'City_Tier 2' * 55.13897430923251  +'Record'772.2334457445644 

* Si es hombre y vive en CT3: 

Total Spend= -79.41713030137271+ 'Monthly Income'0.14753898049205744 + 'Transaction Time'* 0.15494612549589679 +'Gender_Male' * 131.0250132555466 + 'City_Tier 3' *-131.90330031972795 +'Record'772.2334457445644 

* Si es mujer y vive en CT1: 

Total Spend= -79.41713030137271+ 'Monthly Income'0.14753898049205744 + 'Transaction Time'* 0.15494612549589679  + 'Gender_Female'* -131.0250132555465 + City_Tier 1'76.764 + 'Record'772.2334457445644

* Si es mujer y vive en CT2: 

Total Spend= -79.41713030137271+ 'Monthly Income'0.14753898049205744 + 'Transaction Time'* 0.15494612549589679 + 'Gender_Female'* -131.0250132555465 +'City_Tier 2' * 55.13897430923251 + 'Record'772.2334457445644 

* Si es mujer y vive en CT3:

Total Spend= -79.41713030137271+ 'Monthly Income'0.14753898049205744 + 'Transaction Time'* 0.15494612549589679 +'Gender_Female'* -131.0250132555465 + 'City_Tier 3' *-131.90330031972795 +'Record'772.2334457445644 

#  Eliminar variables dummy

In [28]:
dummy_gender = pd.get_dummies(df["Gender"],prefix="Gender").iloc[:,1:]
dummy_gender

,Gender_Male
0,0
1,0
2,1
3,0
4,0
...,...
2357,1
2358,1
2359,0
2360,0


In [31]:
dummy_city_tier = pd.get_dummies(df["City Tier"],prefix="City").iloc[:,1:]
dummy_city_tier.head()

,City_Tier 2,City_Tier 3
0,0,0
1,1,0
2,1,0
3,0,0
4,1,0


In [34]:
column_names = df.columns.values.tolist()
df_new = df[column_names].join(dummy_gender)
column_names = df_new.columns.values.tolist()
df_new = df_new[column_names].join(dummy_city_tier)
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Male,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,0,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,1,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,0,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,0,1,0


In [35]:
features_cols =['Monthly Income',  'Transaction Time','Gender_Male',
                'City_Tier 2','City_Tier 3',"Record"]
x= df_new[features_cols]
y= df_new["Total Spend"]
lm = LinearRegression()
lm.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [36]:
print(lm.intercept_)

-133.67781754642056


In [37]:
list(zip(features_cols,lm.coef_))

[('Monthly Income', 0.14753898049205744),
 ('Transaction Time', 0.15494612549589631),
 ('Gender_Male', 262.05002651109595),
 ('City_Tier 2', -21.62535170126296),
 ('City_Tier 3', -208.66762633022324),
 ('Record', 772.2334457445635)]

In [38]:
lm.score(x,y)

0.9179923586131016

Coeficientes con todas las variables en el modelo
* ('Monthly Income', 0.14753898049205744),
* ('Transaction Time', 0.15494612549589679),
* ('Gender_Male', 131.0250132555466),
* ('Gender_Female', -131.0250132555465),
* ('City_Tier 1', 76.76432601049541),
* ('City_Tier 2', 55.13897430923251),
* ('City_Tier 3', -131.90330031972795),
* ('Record', 772.2334457445644)]



coeficientes eliminando los dummy sobrantes

* ('Monthly Income', 0.14753898049205744),
* ('Transaction Time', 0.15494612549589631),
* ('Gender_Male', 262.05002651109595),
* ('City_Tier 2', -21.62535170126296),
* ('City_Tier 3', -208.66762633022324),
* ('Record', 772.2334457445635)]

Los cambios se reflejan en 
* Gender male, 
    * viene de --131.02 dps--> 262.05=(131.02-(-131.02))
* Gender Female
* CT1
* CT2
* CT3